In [1]:
import os
import whisper
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue # pass messages btw threads

#security credential for python 3.12
os.environ['SSL_CERT_FILE'] = '/etc/ssl/cert.pem' 

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description="Record",
    disable=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disable=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)

    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()

        transcribe = Thread(target=speech_recognition)
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [2]:
import pyaudio 

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'LG ULTRAGEAR', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.009833333333333333, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.019166666666666665, 'defaultSampleRate': 48000.0}
{'index': 1, 'structVersion': 2, 'name': 'MacBook Air Microphone', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.03235416666666667, 'defaultLowOutputLatency': 0.01, 'defaultHighInputLatency': 0.0416875, 'defaultHighOutputLatency': 0.1, 'defaultSampleRate': 48000.0}
{'index': 2, 'structVersion': 2, 'name': 'MacBook Air Speakers', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.024125, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.03345833333333333, 'defaultSampleRate': 48000.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Teams Audio', 'hostApi

In [ ]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2,
                    frames_per_buffer=chunk)
    
    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []
        
    stream.stop_stream()
    stream.close()
    p.terminate()

In [ ]:
import json

model = whisper.load_model("turbo")